In [13]:
import inspect
import os
import sys
from platform import python_version

In [14]:
print(python_version())
inspect.getfile(os)

3.9.5


'/srv/conda/envs/notebook/lib/python3.9/os.py'

In [1]:
# display host information
!ls -laht ~
!pwd
!du -s -h ~
#!df -h
#!lscpu
!free -h
!g++-8 --version

total 2.0M
drwxr-xr-x 1 jovyan jovyan 4.0K Jun 27 22:04 .
drwxr-xr-x 5 jovyan root   4.0K Jun 27 22:04 .ipython
drwxr-xr-x 2 jovyan root   4.0K Jun 27 22:04 .ipynb_checkpoints
drwxr-xr-x 1 jovyan jovyan 4.0K Jun 27 22:04 .jupyter
drwxr-xr-x 3 jovyan jovyan 4.0K Jun 27 21:58 .yarn
drwxr-xr-x 3 jovyan jovyan 4.0K Jun 27 21:57 .npm
drwxr-xr-x 1 jovyan jovyan 4.0K Jun 27 21:57 .cache
drwxr-xr-x 1 jovyan jovyan 4.0K Jun 27 21:57 .local
-rwxr-xr-x 1 jovyan jovyan 1.7M Jun 27 21:57 connection_subgraph_undirected
drwxr-xr-x 1 jovyan jovyan 4.0K Jun 27 21:57 .empty
drwxrwsr-x 1 jovyan jovyan 4.0K Jun 27 21:56 .conda
drwx------ 1 jovyan jovyan 4.0K Jun 27 21:49 .config
drwxr-xr-x 1 root   root   4.0K Jun 27 21:49 ..
-rw-r--r-- 1 jovyan jovyan   38 Jun 27 21:49 apt.txt
-rw-r--r-- 1 jovyan jovyan 128K Jun 27 21:49 connection_subgraph_undirected.cpp
-rw-r--r-- 1 jovyan jovyan  275 Jun 27 21:49 environment.yml
-rw-r--r-- 1 jovyan jovyan  52K Jun 27 21:49 example_06.ipynb
drwxr-xr-x 1 jovyan jovyan 4

In [2]:
import json
import math

from collections import defaultdict
from pprint import pformat

import matplotlib.pyplot as plt

import networkx as nx
from networkx.algorithms import bipartite

import ipywidgets as widgets
from ipywidgets import Output

from IPython.display import display, HTML, IFrame

import ipycytoscape

out = Output()

In [3]:
# # # # # #

In [4]:
class BasicGraph :
    def __init__(self, edge_list_filename = None) :
        self.vertices = list()
        self.edges = list()
        
        if edge_list_filename is not None :
            self.read_edge_list_from_file(edge_list_filename)
        
    def add_vertex(self, vertex) :
        self.vertices.append(vertex)
    
    def add_edge(self, source_vertex, targe_vertex) :
        self.edges.append((source_vertex, targe_vertex))
        
    def read_edge_list_from_file(self, filename) :
        tmp_vertices = set()
        with open (filename, "r") as read_file :
            for line in read_file :
                line = line.strip()
                tokens = line.split(" ")
                s = int(tokens[0].strip(' \t\n\r'))
                t = int(tokens[1].strip(' \t\n\r'))
                self.add_edge(s, t)
                tmp_vertices.add(s)
                tmp_vertices.add(t)
                
        self.vertices.clear()
        self.vertices.extend(sorted(tmp_vertices))
        
    def __str__(self) :
        return "Vertices(" + str(len(self.vertices)) + "): " + str(self.vertices) + "\n" + \
    "Edges(" + str(len(self.edges)) + "): " + str(self.edges)    

In [5]:
# Test
basic_graph = BasicGraph()

basic_graph.add_vertex(0)
basic_graph.add_vertex(1)
basic_graph.add_vertex(2)

basic_graph.add_edge(0, 1)
basic_graph.add_edge(0, 2)
basic_graph.add_edge(1, 0)
basic_graph.add_edge(1, 2)
basic_graph.add_edge(2, 0)
basic_graph.add_edge(2, 1)

#print(basic_graph)

In [6]:
# # # # # #

In [7]:
# large data files
data_file_url_list = ["https://www.ece.ubc.ca/~treza/data/vertex_data", "https://www.ece.ubc.ca/~treza/data/edge_list", \
                      "https://www.ece.ubc.ca/~treza/data/vertex_data_json"]

In [10]:
# download large data files
for file_url in data_file_url_list :
    print(file_url)
    !wget --tries=5 --continue {file_url} --directory-prefix ~ # uncomment

https://www.ece.ubc.ca/~treza/data/vertex_data
--2021-06-27 22:05:10--  https://www.ece.ubc.ca/~treza/data/vertex_data
Resolving www.ece.ubc.ca (www.ece.ubc.ca)... 206.87.224.50
Connecting to www.ece.ubc.ca (www.ece.ubc.ca)|206.87.224.50|:443... connected.
HTTP request sent, awaiting response... 307 Temporary Redirect
Location: https://people.ece.ubc.ca/treza/data/vertex_data [following]
--2021-06-27 22:05:11--  https://people.ece.ubc.ca/treza/data/vertex_data
Resolving people.ece.ubc.ca (people.ece.ubc.ca)... 142.103.83.164
Connecting to people.ece.ubc.ca (people.ece.ubc.ca)|142.103.83.164|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 82868159 (79M)
Saving to: ‘/home/jovyan/vertex_data’

vertex_data         100%[===================>]  79.03M  38.3MB/s    in 2.1s    

2021-06-27 22:05:13 (38.3 MB/s) - ‘/home/jovyan/vertex_data’ saved [82868159/82868159]

https://www.ece.ubc.ca/~treza/data/edge_list
--2021-06-27 22:05:14--  https://www.ece.ubc.ca/~treza/data/

In [15]:
# input/ouput files
edge_list_filename = os.path.expanduser("~/edge_list")
vertex_data_filename = os.path.expanduser("~/vertex_data")
vertex_data_json_filename = os.path.expanduser("~/vertex_data_json")
output_filename = os.path.expanduser("~/output_05")

In [16]:
# # # # # #

In [17]:
# vertex data
include_json_string = True

vertex_type_map = dict()
vertex_data_map = defaultdict(tuple)

def parse_json_data_2(json_data, line, include_json_string = False) :
    id_int = int(json_data["id"])
    label = json_data["labels"][0]
    properties_identifier = ""
    properties_name = ""
    
    if "properties" in json_data :
        if "identifier" in json_data["properties"] :
            properties_identifier = json_data["properties"]["identifier"]
        if "name" in json_data["properties"] :
            properties_name = json_data["properties"]["name"][:15]
    
    # Test
    # to save memory
    if include_json_string == False :
        line = ""
        
    vertex_data_map[id_int] = (label, properties_identifier, properties_name, line)
    
    vertex_type_count = vertex_type_map.get(label, 0)
    vertex_type_map[label] = vertex_type_count + 1
    
with open(vertex_data_json_filename, "r") as read_file :
    line_count = 0;
    for line in read_file :
        line = line.strip()
        try :
            json_data = json.loads(line)
            parse_json_data_2(json_data, line, include_json_string)
        except Exception as e :
            print(e)
            continue
        line_count = line_count + 1
    #print(str(line_count))

print(len(vertex_data_map))
print(len(vertex_type_map))

2267796
22


In [18]:
# vertex data
def get_vertex_type_count_list(vertex_type_map) :
    tuple_list = []
    for k, v in vertex_type_map.items() :
        tuple_list.append((str(k) + " (" + str(v) + ")", k))
    return tuple_list
    #print(tuple_list)
    
#vertex_type_count_list()

In [19]:
# vertex data
def vertices_filtered_by_type(vertex_type, begin = 0, end = 25) :
    #filtered_veretx_data_map = dict()
    filtered_veretx_data_list = []
    for k, v in vertex_data_map.items() :
        if v[0] != vertex_type :
            continue
        else :
            #filtered_veretx_data_map[k] = v
            #filtered_veretx_data_map[(v, k)] = v
            
            if v[2] != "" :
                #filtered_veretx_data_map[k] = v[2]
                filtered_veretx_data_list.append((str(k) + " (" + str(v[2]) + ")", k))
            elif v[1] != "" :
                #filtered_veretx_data_map[k] = v[1]
                filtered_veretx_data_list.append((str(k) + " (" + str(v[1]) + ")", k))
            else :
                #filtered_veretx_data_map[k] = str(k) #v[0]
                filtered_veretx_data_list.append((str(k) + " (" + str(v[2]) + ")", k))
                
    #return filtered_veretx_data_map
    #return filtered_veretx_data_list

    if (end > len(filtered_veretx_data_list)) :
        end = len(filtered_veretx_data_list)
    return filtered_veretx_data_list[begin : end], end, len(filtered_veretx_data_list)

#print(len(vertices_filtered_by_type("Organism")))
#print(vertices_filtered_by_type("Organism")[0:10])
x,y,z = vertices_filtered_by_type("Organism", 15, 150)
print(len(x))
print(z)

135
8109


In [20]:
# search
import re

def kw_search_filtered_by_veretx_type(vertex_type_set, keyword, result_list) :
    for k, v in vertex_data_map.items() :        
        if v[0] not in vertex_type_set :
            continue
        match_found = False
        
        complete_json_string = v[3]
        
        match_found = re.search(keyword, complete_json_string, re.IGNORECASE)
        
        if match_found :
            #print(v[3])
            if v[2] != "" :                
                result_list.append((str(k) + " (" + str(v[2]) + ")", k))
            elif v[1] != "" :                
                result_list.append((str(k) + " (" + str(v[1]) + ")", k))
            else :                
                result_list.append((str(k) + " (" + str(v[2]) + ")", k))               
    
    return len(result_list)

tmp_list = list()
#print(kw_search_filtered_by_veretx_type({"Organism", "EC"}, "Carbon", tmp_list))
print(kw_search_filtered_by_veretx_type({"Food"}, "apple", tmp_list))
#print(kw_search_filtered_by_veretx_type({"Organism"})[0:10])
tmp_list.clear()

9


In [21]:
def load_more_from_list_by_range(item_list, begin = 0, end = 25) :
    if (end > len(item_list)) :
        end = len(item_list)
    return item_list[begin : end], end, len(item_list)

In [22]:
# # # # # #

In [23]:
# input widgets
vertex_type_count_list = get_vertex_type_count_list(vertex_type_map)

vertex_type_selected_a = set()
vertex_type_selected_b = set()

vertex_type_a_set = set() # TODO: change name, the set contains vertex IDs
vertex_type_b_set = set() # TODO: cange name
vertex_type_c_set = set()

sub_drdn_btn_pair_map = defaultdict(list)    

hbox_vertex_data_a = widgets.HBox([])
hbox_vertex_data_b = widgets.HBox([])

label_selected_vertices_a = widgets.Label("Empty")
label_selected_vertices_b = widgets.Label("Empty")
label_selected_vertex_types = widgets.Label("Empty")

txar_vertex_info = widgets.Textarea(value="", description="Vertex info", placeholder="Empty", disabled=True, 
                                    layout=widgets.Layout(width='30%', height='200px', align_content='center'))

# search
load_more_count = 50

search_result_a_list = list()
search_result_b_list = list()

text_kw_search_a = widgets.Text(value="", placeholder="Empty")
text_kw_search_b = widgets.Text(value="", placeholder="Empty")

label_kw_search_result_count_a = widgets.Label("")
label_kw_search_result_count_b = widgets.Label("")

# # # # # #

def txar_vertex_info_change(vertex) :
    if str(vertex_data_map[int(vertex)][3]) != "" :
        txar_vertex_info.value = str(vertex_data_map[int(vertex)][3]) # json string
    else :
        txar_vertex_info.value = str(vertex_data_map[int(vertex)])        
    #print(vertex_data_map[int(vertex)])

def btn_reset_all_on_click(*args) :
    vertex_type_selected_a.clear()
    vertex_type_selected_b.clear()
    
    vertex_type_a_set.clear()
    vertex_type_b_set.clear()
    vertex_type_c_set.clear()
  
    hbox_vertex_data_a.children = tuple()
    hbox_vertex_data_b.children = tuple()

    txar_vertex_info.value = "Empty"
    
    label_selected_vertices_a.value = "Empty"
    label_selected_vertices_b.value = "Empty"
    label_selected_vertex_types.value = "Empty"
    
    sub_drdn_btn_pair_map.clear()
    
    text_kw_search_a.value = ""
    text_kw_search_b.value = ""
    
    label_kw_search_result_count_a.value = ""
    label_kw_search_result_count_b.value = ""
    
    search_result_a_list.clear()
    search_result_b_list.clear()
    
def btn_selected_vertices_a_on_click(*args) :
    # args[0] is the object itself
    vertex_type_a_set.clear()
    label_selected_vertices_a.value = "Empty"    
    txar_vertex_info.value = "Empty"

def btn_selected_vertices_b_on_click(*args) :
    vertex_type_b_set.clear()
    label_selected_vertices_b.value = "Empty"    
    txar_vertex_info.value = "Empty"

def btn_selected_vertex_types_on_click(*args) :
    vertex_type_c_set.clear()
    label_selected_vertex_types.value = "Empty"    
        
def on_sub_drdn_value_change_a(change) :
    #print("a: " + str(change["type"]))
    #print("a: " + str(change["new"]))
    #print("a: " + str(len(change["new"])))
    
    if len(change["new"]) < 1 :
        txar_vertex_info.value = "Empty"
        return
    
    for i in change["new"] :
        vertex_type_a_set.add(i)
    #print(vertex_type_a_set)    
    label_selected_vertices_a.value = str(sorted(vertex_type_a_set))
    txar_vertex_info_change(change["new"][0])
    
def on_sub_drdn_value_change_b(change) :
    #print("b: " + str(change["new"]))
    
    if len(change["new"]) < 1 :
        txar_vertex_info.value = "Empty"
        return
    
    for i in change["new"] :
        vertex_type_b_set.add(i)
    #print(vertex_type_b_set)
    label_selected_vertices_b.value = str(sorted(vertex_type_b_set))
    txar_vertex_info_change(change["new"][0])

def btn_search_result_load_more_on_click(*args) :
    btn = args[0]
    #print(id(btn))
    #print(sub_drdn_btn_pair_map)
    
    if sub_drdn_btn_pair_map[id(btn)][1] >= sub_drdn_btn_pair_map[id(btn)][2] :
        return
    
    drdn_found = False
    
    for vbox in hbox_vertex_data_a.children :
        for child in vbox.children : 
            if sub_drdn_btn_pair_map[id(btn)][0] == id(child) :
                #print(sub_drdn_btn_pair_map)
        
                vertex_type = sub_drdn_btn_pair_map[id(btn)][3]
                begin = int(sub_drdn_btn_pair_map[id(btn)][1]) # starting point for this batch
                end = begin + load_more_count # (begin : end]
                
                #print(vertex_type)
                #print(begin)
                #print(str(end - 1)) # (begin : end-1)
                #print(len(search_result_a_list))
                
                sub_drdn_options_list, sub_drdn_options_list_end, sub_drdn_options_list_len = \
                    load_more_from_list_by_range(search_result_a_list, begin, end)
                
                #print(len(sub_drdn_options_list))
                #print(sub_drdn_options_list_end)
                #print(sub_drdn_options_list_len)
                
                child.options = sub_drdn_options_list # replace the previous entries
                
                sub_drdn_btn_pair_map[id(btn)][1] = sub_drdn_options_list_end # starting point for the next batch                
                                
                btn.description = "Load more " + str(begin + 1) + "-" + str(sub_drdn_options_list_end)            
             
                drdn_found = True
                break
            if drdn_found :
                break
                
    for vbox in hbox_vertex_data_b.children :
        for child in vbox.children : 
            if sub_drdn_btn_pair_map[id(btn)][0] == id(child) :
                        
                vertex_type = sub_drdn_btn_pair_map[id(btn)][3]
                begin = int(sub_drdn_btn_pair_map[id(btn)][1]) # starting point for this batch
                end = begin + load_more_count # (begin : end]
                                                
                sub_drdn_options_list, sub_drdn_options_list_end, sub_drdn_options_list_len = \
                    load_more_from_list_by_range(search_result_b_list, begin, end)
                                                
                child.options = sub_drdn_options_list # replace the previous entries
                sub_drdn_btn_pair_map[id(btn)][1] = sub_drdn_options_list_end # starting point for the next batch                                
                btn.description = "Load more " + str(begin + 1) + "-" + str(sub_drdn_options_list_end)            
             
                drdn_found = True
                break
            if drdn_found :
                break

def btn_kw_search_a_on_click(*args) :
    hbox_vertex_data_a.children = tuple()
    label_kw_search_result_count_a.value = ""
    label_selected_vertices_a.value = "Empty"
    txar_vertex_info.value = "Empty"    
    search_result_a_list.clear()
    
    if text_kw_search_a.value == "" :
        return
    vertex_type = "dummy" # TODO: remove
    #print(text_kw_search_a.value)    
    #print(vertex_type_selected_a)
    kw_search_filtered_by_veretx_type(vertex_type_selected_a, text_kw_search_a.value, search_result_a_list)
    label_kw_search_result_count_a.value = str(len(search_result_a_list))
    #print(len(search_result_a_list))
    
    sub_drdn_options_list, sub_drdn_options_list_end, sub_drdn_options_list_len = \
        load_more_from_list_by_range(search_result_a_list, 0, load_more_count)
    
    sub_drdn = widgets.SelectMultiple(options=sub_drdn_options_list, description="")        
    sub_drdn.observe(on_sub_drdn_value_change_a, names='value')
    
    sub_btn = widgets.Button(description="Load more 1-" + str(sub_drdn_options_list_end), tooltip="Tooltip", disabled=False)
    sub_btn.on_click(callback=btn_search_result_load_more_on_click)
               
    vbox_sub_drdn_btn = widgets.VBox([sub_drdn, sub_btn])
    
    #hbox_vertex_data_a.children = hbox_vertex_data_a.children + (sub_drdn,)
    hbox_vertex_data_a.children = hbox_vertex_data_a.children + (vbox_sub_drdn_btn,)
    
    sub_drdn_btn_pair_map[id(sub_btn)] = [id(sub_drdn), sub_drdn_options_list_end, sub_drdn_options_list_len, vertex_type]

def btn_kw_search_b_on_click(*args) :
    hbox_vertex_data_b.children = tuple()
    label_kw_search_result_count_b.value = ""
    label_selected_vertices_b.value = "Empty"
    txar_vertex_info.value = "Empty"    
    search_result_b_list.clear()
    
    if text_kw_search_b.value == "" :
        return
    vertex_type = "dummy" # TODO: remove
    #print(text_kw_search_b.value)    
    #print(vertex_type_selected_b)
    kw_search_filtered_by_veretx_type(vertex_type_selected_b, text_kw_search_b.value, search_result_b_list)
    label_kw_search_result_count_b.value = str(len(search_result_b_list))
    #print(len(search_result_b_list))
    
    sub_drdn_options_list, sub_drdn_options_list_end, sub_drdn_options_list_len = \
        load_more_from_list_by_range(search_result_b_list, 0, load_more_count)
    
    sub_drdn = widgets.SelectMultiple(options=sub_drdn_options_list, description="")        
    sub_drdn.observe(on_sub_drdn_value_change_b, names='value')
    
    sub_btn = widgets.Button(description="Load more 1-" + str(sub_drdn_options_list_end), tooltip="Tooltip", disabled=False)
    sub_btn.on_click(callback=btn_search_result_load_more_on_click)
               
    vbox_sub_drdn_btn = widgets.VBox([sub_drdn, sub_btn])
    
    #hbox_vertex_data_b.children = hbox_vertex_data_b.children + (sub_drdn,)
    hbox_vertex_data_b.children = hbox_vertex_data_b.children + (vbox_sub_drdn_btn,)
    
    sub_drdn_btn_pair_map[id(sub_btn)] = [id(sub_drdn), sub_drdn_options_list_end, sub_drdn_options_list_len, vertex_type]
    
def on_drdn_value_change_a(change) :
    #print(change["new"])    
    #print(change)
    hbox_vertex_data_a.children = tuple()
    text_kw_search_a.value = ""
    label_kw_search_result_count_a.value = ""
    label_selected_vertices_a.value = "Empty"
    txar_vertex_info.value = "Empty"    
    vertex_type_selected_a.clear()
    vertex_type_a_set.clear()
    
    for i in change["new"] :
        vertex_type = str(i)        
        vertex_type_selected_a.add(vertex_type)              
    
def on_drdn_value_change_b(change) :
    #print(change["new"])    
    #print(change)
    hbox_vertex_data_b.children = tuple()    
    text_kw_search_b.value = ""
    label_kw_search_result_count_b.value = ""
    label_selected_vertices_b.value = "Empty"
    txar_vertex_info.value = "Empty"    
    vertex_type_selected_b.clear()
    vertex_type_b_set.clear()
    
    for i in change["new"] :
        vertex_type = str(i)
        vertex_type_selected_b.add(vertex_type)       
        
def on_drdn_value_change_c(change) :
        #print(change["new"])  
        for i in change["new"] :
            vertex_type_c_set.add(i)
        label_selected_vertex_types.value = str(sorted(vertex_type_c_set))

# # # # # #
        
#drdn_vertex_type_a = widgets.SelectMultiple(options=[*vertex_type_map], description="Source types")
drdn_vertex_type_a = widgets.SelectMultiple(options=vertex_type_count_list, description="Source types")
drdn_vertex_type_a.observe(on_drdn_value_change_a, names='value')

#drdn_vertex_type_b = widgets.SelectMultiple(options=[*vertex_type_map], description="Target types")
drdn_vertex_type_b = widgets.SelectMultiple(options=vertex_type_count_list, description="Target types")
drdn_vertex_type_b.observe(on_drdn_value_change_b, names='value')

btn_kw_search_a = widgets.Button(description="Search")
btn_kw_search_a.on_click(callback=btn_kw_search_a_on_click)

btn_kw_search_b = widgets.Button(description="Search")
btn_kw_search_b.on_click(callback=btn_kw_search_b_on_click)

btn_selected_vertices_a = widgets.Button(description="Reset", tooltip="Tooltip", disabled=False)
btn_selected_vertices_a.on_click(callback=btn_selected_vertices_a_on_click)

btn_selected_vertices_b = widgets.Button(description="Reset", tooltip="Tooltip", disabled=False)
btn_selected_vertices_b.on_click(callback=btn_selected_vertices_b_on_click)

btn_reset_all = widgets.Button(description="Reset", disabled=False)
btn_reset_all.on_click(callback=btn_reset_all_on_click)

hbox_vertex_type_ab = widgets.HBox([drdn_vertex_type_a, drdn_vertex_type_b, txar_vertex_info, btn_reset_all])

# filter by vertex type
drdn_vertex_type_c = widgets.SelectMultiple(options=vertex_type_count_list, description="Filter by vertex type")
drdn_vertex_type_c.observe(on_drdn_value_change_c, names='value')

btn_selected_vertex_types = widgets.Button(description="Reset", disabled=False)
btn_selected_vertex_types.on_click(callback=btn_selected_vertex_types_on_click)

#print(widgets.Widget.observe.__doc__)

In [24]:
# display input widgets
display(hbox_vertex_type_ab)

display(widgets.Label("Source vertices"))
display(widgets.HBox([btn_kw_search_a, text_kw_search_a, label_kw_search_result_count_a]))
display(hbox_vertex_data_a)

display(widgets.HBox([btn_selected_vertices_a, widgets.Label("Selected source vertices"), label_selected_vertices_a]))

display(widgets.Label("Target vertices"))
display(widgets.HBox([btn_kw_search_b, text_kw_search_b, label_kw_search_result_count_b]))
display(hbox_vertex_data_b)
display(widgets.HBox([btn_selected_vertices_b, widgets.Label("Selected target vertices"), label_selected_vertices_b]))

display(widgets.HBox([drdn_vertex_type_c, btn_selected_vertex_types, widgets.Label("Selected vertex types"),  
                      label_selected_vertex_types]))

Label(value='Source vertices')

HBox()

Label(value='Target vertices')

HBox()

In [ ]:
# # # # # #

In [25]:
# input parameters
print(vertex_type_a_set)
print(vertex_type_b_set)
print(vertex_type_c_set)
# TODO

{2268378}
{2283541}
{'Reaction', 'EC', 'Compound', 'Organism'}


In [26]:
vertex_type_c_map = dict()

with open (vertex_data_filename, "r") as read_file :
    for line in read_file :
        if len(vertex_type_c_set) < 1 :
            break
        line = line.strip()
        tokens = line.split(" ")
        vertex = int(tokens[0].strip(' \t\n\r'))
        vertex_data_hash = int(tokens[1].strip(' \t\n\r'))
        vertex_data_string = str(tokens[2].strip(' \t\n\r'))        
        if vertex_data_string in vertex_type_c_map :
            continue
        if vertex_data_string in vertex_type_c_set :
            vertex_type_c_map[vertex_data_string] = vertex_data_hash
        if len(vertex_type_c_map) == len(vertex_type_c_set) :
            break

if len(vertex_type_c_map) != len(vertex_type_c_set) :
    print("Error: items not found.")
print(vertex_type_c_map)        

{'Compound': 1546094233688003844, 'EC': 8121734289560993151, 'Organism': 11838663228819696432, 'Reaction': 13996733750220222916}


In [ ]:
# # # # # #

In [30]:
# compute paths
s_t_vertices = []
#s_t_vertices = [0, 1]
#search_depth = 3
# command

In [31]:
# # # # # #

In [32]:
# create command string
command_string_vertex_data_filter = ""
for k, v in vertex_type_c_map.items() :
    command_string_vertex_data_filter = command_string_vertex_data_filter + " -f " + str(v)
print(command_string_vertex_data_filter)

 -f 1546094233688003844 -f 8121734289560993151 -f 11838663228819696432 -f 13996733750220222916


In [33]:
# create command string
command_string = ""
for i in vertex_type_a_set :
    command_string = command_string + " -u " + str(i) + " "
    s_t_vertices.append(i)

for i in vertex_type_b_set :
    command_string = command_string + " -v " + str(i) + " "
    s_t_vertices.append(i)

for k, v in vertex_type_c_map.items() :
    command_string = command_string + " -f " + str(v) + " "
print(command_string)

 -u 2268378  -v 2283541  -f 1546094233688003844  -f 8121734289560993151  -f 11838663228819696432  -f 13996733750220222916 


In [ ]:
s_t_vertices = [2268378, 2283541]
search_depth = 4
!./connection_subgraph_undirected ~/edge_list ~/vertex_data ~/tmp -d 4 -u 2268378 -v 2283541 2> ~/output_05

In [ ]:
s_t_vertices = [2268378, 2283541]
search_depth = 5
!./connection_subgraph_undirected {edge_list_filename} {vertex_data_filename} ~/tmp -d {search_depth} -u {s_t_vertices[0]} -v {s_t_vertices[1]} 2> {output_filename}

In [ ]:
s_t_vertices = [2268378, 2283541]
search_depth = 4
!./connection_subgraph_undirected ~/edge_list ~/vertex_data ~/tmp -d 4 -u 2268378 -v 2283541 -f 1546094233688003844 -f 13996733750220222916 2> ~/output_05

In [ ]:
s_t_vertices = [2268378, 2283541]
search_depth = 6
!./connection_subgraph_undirected {edge_list_filename} {vertex_data_filename} ~/tmp -d {search_depth} -u {s_t_vertices[0]} -v {s_t_vertices[1]} {command_string_vertex_data_filter} 2> {output_filename}

In [34]:
# UI to command string example
#s_t_vertices = [2268378, 2283541]
search_depth = 5
command_string = command_string + " -d " + str(search_depth) + " " 
!./connection_subgraph_undirected {edge_list_filename} {vertex_data_filename} ~/tmp {command_string}  2> {output_filename}

CMD line:  ./connection_subgraph_undirected /home/jovyan/edge_list /home/jovyan/vertex_data /home/jovyan/tmp -u 2268378 -v 2283541 -f 1546094233688003844 -f 8121734289560993151 -f 11838663228819696432 -f 13996733750220222916 -d 5

Hostname: jupyter-llnl-2dhavoqgt-2d9hpty0u3
Number of available threads on jupyter-llnl-2dhavoqgt-2d9hpty0u3: 6

Building in-memory CSR graph (undirected) ...
Reading edge list file ...
Size of edge list: 36226384
Max vertex: 2329608
Sorting edges by source vertex ...
Creating CSR row pointers ...
Size of vertex list: 2329610
Size of vertex degree list: 2329609
Sorting neighbors ...
Generating edge list ...
Size of edge list: 36226384
CSR Graph generation completed.
Number of vertices: 2329609
Number of edges: 36226384
Max vertex: 2329608
Reading vertex data file ...
Size of vertex data list: 2329609
Size of edge state list: 36226384, 2
Time: 34.0369 seconds.

Selected veretx data: 11838663228819696432, 13996733750220222916, 1546094233688003844, 8121734289560

In [ ]:
s_t_vertices = [1971037, 1971579] # not found
search_depth = 20
!./connection_subgraph_undirected {edge_list_filename} {vertex_data_filename} ~/tmp -d {search_depth} -u {s_t_vertices[0]} -v {s_t_vertices[1]} {command_string_vertex_data_filter} 2> {output_filename}

In [ ]:
s_t_vertices = [53684, 1970914] # not found
search_depth = 20
!./connection_subgraph_undirected ~/edge_list ~/vertex_data ~/tmp -d 4 -u 53684 -v 1970914 -f 1546094233688003844 -f 8121734289560993151 -f 11838663228819696432 -f 13996733750220222916 2> ~/output_05

In [ ]:
# # # # # #

In [35]:
#output_graph = basic_graph
output_graph = BasicGraph(output_filename)
print("Vertex count: " + str(len(output_graph.vertices)))
print("Edge count: " + str(len(output_graph.edges)))

Vertex count: 11
Edge count: 24


In [36]:
# cytoscape widgets

In [37]:
def basic_graph_to_cytoscape_graph (graph) :
    graph_json = {"nodes" : [], "edges" : []}
   
    for v in graph.vertices :
            #print(v)
            v_dict = {"data" : {}}
            v_dict["data"]["id"] = str(v)
            v_dict["data"]["idInt"] = v
            #v_dict["data"]["label"] = str(v)
            v_dict["data"]["type"] = str(vertex_data_map[v][0])
            v_dict["data"]["label"] = str(vertex_data_map[v][2])
            v_dict["data"]["description"] = str(vertex_data_map[v][3])
            graph_json["nodes"].append(v_dict)
        
    for s, t in graph.edges :
            #print(s, end = " ")
            #print(t)
            e_dict = {"data" : {}}
            #e_dict["data"]["id"] = ""
            e_dict["data"]["source"] = str(s)
            e_dict["data"]["target"] = str(t)
            graph_json["edges"].append(e_dict)

    #print(graph_json)
    return graph_json

#basic_graph_edge_list_to_cytoscape_graph (output_graph)

In [38]:
node_info_widget = widgets.Textarea(value="", placeholder="Information", disabled=True, 
                                   layout=widgets.Layout(width='50%', height='200px', align_content='center'))

In [39]:
cytoscape_graph_json = basic_graph_to_cytoscape_graph (output_graph)
cytoscape_object = ipycytoscape.CytoscapeWidget()
cytoscape_object.graph.add_graph_from_json(cytoscape_graph_json)

#cytoscape_object.set_layout(name='grid') # broken
cytoscape_object.set_layout(name="cola")
#cytoscape_object.set_layout(name="klay")
#cytoscapeobj.set_layout(nodeSpacing=100)
#cytoscapeobj.get_layout()

#cytoscape_object.set_tooltip_source("label")
cytoscape_object.set_tooltip_source("description")

In [40]:
def cytoscape_graph_set_style () :
    cytoscape_object.set_style([
    {
      "selector": "edge.highlighted",
      "css": {
        "line-color": "red"
      }
    },
    {
      "selector": "node.highlighted",
      "css": {
        "background-color": "yellow",
        "border-color": "black",
        "border-width": "5px",
        "border-opacity": "0.3",  
      },
    },
    {
      "selector": "node.Anatomy",
      "css": {
        "background-color": "rgb(179, 222, 105)"
      },
    },    
    {
      "selector": "node.BiologicalProcess",
      "css": {
        "background-color": "rgb(253, 180, 98)"
      },
    },
    {
      "selector": "node.CellularComponent",
      "css": {
        "background-color": "rgb(255, 255, 179)"
      },
    },        
    {
      "selector": "node.Compound",
      "css": {
        "background-color": "rgb(188, 128, 189)"
      },
    },
    {
      "selector": "node.Disease",
      "css": {
        "background-color": "rgb(251, 128, 114)"
      },
    },       
    {
      "selector": "node.EC",
      "css": {
        "background-color": "rgb(247, 151, 103)"
      },
    },  
    {
      "selector": "node.Food",
      "css": {
        "background-color": "rgb(199, 183, 143)"
      },
    },      
    {
      "selector": "node.Gene",
      "css": {
        "background-color": "rgb(128, 177, 211)"
      },
    },
    {
      "selector": "node.MolecularFunction",
      "css": {
        "background-color": "rgb(255, 237, 111)"
      },
    },    
    {
      "selector": "node.Organism",
      "css": {
        "background-color": "rgb(217, 200, 174)"
      },
    },    
    {
      "selector": "node.Pathway",
      "css": {
        "background-color": "rgb(87, 199, 227)"
      },
    },
    {
      "selector": "node.PharmacologicClass",
      "css": {
        "background-color": "rgb(190, 186, 218)"
      },
    }, 
    {
      "selector": "node.Protein",
      "css": {
        "background-color": "rgb(141, 211, 199)"
      },
    },     
    {
      "selector": "node.Reaction",
      "css": {
        "background-color": "rgb(241, 102, 103)"
      },
    },
    {
      "selector": "node.SideEffect",
      "css": {
        "background-color": "rgb(204, 235, 197)"
      },
    },
    {
      "selector": "node.Symptom",
      "css": {
        "background-color": "rgb(252, 205, 229)"
      },
    },
    {
      "selector": "node.CellType",
      "css": {
        "background-color": "green"
      },
    },
    {
      "selector": "node.Nutrient",
      "css": {
        "background-color": "cyan"
      },
    },
    {
      "selector": "node.MolecularFunction",
      "css": {
        "background-color": "darkcyan"
      },
    },    
    {
      "selector": "node.SARSCov2",
      "css": {
        "background-color": "crimson"
      },
    },
    {
      "selector": "node.DatabaseTimestamp",
      "css": {
        "background-color": "darkgrey"
      },
    },
    {
      "selector": "node.Version",
      "css": {
        "background-color": "darkgrey"
      },
    },    
    {"selector": "node", "style": {"content": "data(label)", "text-valign": "center", "text-halign": "center", "font-size": "8px"}}, 
        {"selector": "node[classes]", "style": {"label": "data(label)"}}])
    
#"background-color": "grey", "content" ...    

In [41]:
def set_node_color_by_type (target_node, node_type) :
#def highlight_node (target_node) :           
    for node in cytoscape_object.graph.nodes:
        if int(node.data["id"]) == int(target_node["data"]["id"]) :
            classes = set(node.classes.split(" "))    
            classes.add(node_type)
            node.classes = " ".join(classes)
            break

def set_node_color() :
    for node in cytoscape_object.graph.nodes :        
        #print(node.data["id"])
        #print(node.data["type"])
        set_node_color_by_type ({"data" : {"id": node.data["id"]}}, node.data["type"])
    #set_node_color_by_type ({"data" : {"id": "2276338"}}, "compound")
    #set_node_color_by_type ({"data" : {"id": "2293366"}}, "reaction")


In [42]:
def unhighlight_nodes () :
    for node in cytoscape_object.graph.nodes:
        classes = set(node.classes.split(" "))
        if "highlighted" in classes :
            classes.remove("highlighted")
            node.classes = " ".join(classes)            

def unhighlight_edges () :
    for edge in cytoscape_object.graph.edges:
        classes = set(edge.classes.split(" "))
        if "highlighted" in classes :
            classes.remove("highlighted")
            edge.classes = " ".join(classes)                

def highlight_node (target_node) :           
    for node in cytoscape_object.graph.nodes:
        if int(node.data["id"]) == int(target_node["data"]["id"]) :
            classes = set(node.classes.split(" "))    
            classes.add("highlighted")
            node.classes = " ".join(classes)
            break
            
def highlight_edges (edge_filter) :
    for edge in cytoscape_object.graph.edges:
        #print(edge.data)
        #print(edge_filter)
    
        s = int(edge.data["source"]) 
        t = int(edge.data["target"])             
    
        if (s, t) in edge_filter :
            classes = set(edge.classes.split(" "))
            classes.add("highlighted")
            #classes.remove("highlighted")            
            edge.classes = " ".join(classes)
            continue
        
        #classes = set(edge.classes.split(" "))
        #classes.add("highlighted")
        #edge.classes = " ".join(classes)
    
        #print(edge)    

In [43]:
def update_node_info_widget(node) :
    #node_info_widget.value = node["data"]["label"] 
    vertex = int(node["data"]["id"])
    #vertex_data = dict(sorted(vertex_data_map[vertex].items()))
    if str(vertex_data_map[vertex][3]) != "" :
        vertex_data = vertex_data_map[vertex][3]
    else :
        vertex_data = vertex_data_map[vertex]
    node_info_widget.value = str(vertex_data)
    #output_string = ""
    #for k, v in vertex_data.items() :
    #    output_string = output_string + str(k) + " : " + str(v) + "\n"
    #node_info_widget.value = output_string

In [44]:
selected_vertex = [0] # must be a container or a class object 

def node_callback_click(node) :
    selected_vertex[0] = int(node["data"]["id"])         
    update_node_info_widget(node)

cytoscape_object.on('node', 'click', node_callback_click)

In [45]:
#highlight_node({"data" : {"id": s_t_vertices[0]}})
#highlight_node({"data" : {"id": s_t_vertices[1]}})
set_node_color()
for i in range(len(s_t_vertices)) :
    highlight_node({"data" : {"id": s_t_vertices[i]}})
cytoscape_graph_set_style()
display(cytoscape_object)
display(node_info_widget)

CytoscapeWidget(cytoscape_layout={'name': 'cola'}, cytoscape_style=[{'selector': 'edge.highlighted', 'css': {'…

Textarea(value='', disabled=True, layout=Layout(align_content='center', height='200px', width='50%'), placehol…